In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore") 

In [2]:
df = pd.read_csv('insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [3]:
df.shape

(1338, 7)

In [4]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


In [6]:
columns = ['age','bmi','children']

In [7]:
for column in columns:
    upper_limit = df[column].mean() + 3*df[column].std()
    lower_limit = df[column].mean() - 3*df[column].std()
    diabetes_df =  df[(df[column]<upper_limit) & (df[column]>lower_limit)]
df.shape

(1338, 7)

In [8]:
categorical_columns = ['sex','children', 'smoker', 'region']
for col in categorical_columns:
    df = pd.get_dummies(data = df, prefix = col, prefix_sep='_',
                               columns = [col],
                               drop_first =True,
                               dtype='int8')

In [9]:
df.head()

,age,bmi,charges,sex_male,children_1,children_2,children_3,children_4,children_5,smoker_yes,region_northwest,region_southeast,region_southwest
0,19,27.900,16884.92400,0,0,0,0,0,0,1,0,0,1
1,18,33.770,1725.55230,1,1,0,0,0,0,0,0,1,0
2,28,33.000,4449.46200,1,0,0,1,0,0,0,0,1,0
3,33,22.705,21984.47061,1,0,0,0,0,0,0,1,0,0
4,32,28.880,3866.85520,1,0,0,0,0,0,0,1,0,0


In [10]:
df.shape

(1338, 13)

## Using sklearn

In [11]:
from sklearn.model_selection import train_test_split
X = df.drop('charges',axis=1)
y = df['charges']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=2023)

In [12]:
linreg = LinearRegression().fit(X_train, y_train)

y_pred = linreg.predict(X_test)

In [13]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

4202.206759734044

## Using tensorflow

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(12,)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='linear')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
    loss='mae')

In [16]:
model.fit(X_train, y_train, epochs = 100)

Epoch 1/100
30/30 [==============================] - 1s 2ms/step - loss: 8562.5918
Epoch 2/100
30/30 [==============================] - 0s 2ms/step - loss: 6727.5508
Epoch 3/100
30/30 [==============================] - 0s 2ms/step - loss: 6299.9014
Epoch 4/100
30/30 [==============================] - 0s 2ms/step - loss: 5858.7031
Epoch 5/100
30/30 [==============================] - 0s 2ms/step - loss: 4698.6055
Epoch 6/100
30/30 [==============================] - 0s 2ms/step - loss: 4401.6729
Epoch 7/100
30/30 [==============================] - 0s 2ms/step - loss: 4202.3491
Epoch 8/100
30/30 [==============================] - 0s 2ms/step - loss: 4027.5549
Epoch 9/100
30/30 [==============================] - 0s 2ms/step - loss: 3809.6848
Epoch 10/100
30/30 [==============================] - 0s 2ms/step - loss: 3781.6753
Epoch 11/100
30/30 [==============================] - 0s 2ms/step - loss: 3675.2808
Epoch 12/100
30/30 [==============================] - 0s 2ms/step - loss: 3642.0090
E

30/30 [==============================] - 0s 2ms/step - loss: 2299.1628
Epoch 99/100
30/30 [==============================] - 0s 2ms/step - loss: 2437.8386
Epoch 100/100
30/30 [==============================] - 0s 2ms/step - loss: 2401.6589


In [17]:
model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 2ms/step - loss: 2628.4839


2628.48388671875

In [18]:
y_pred = model.predict(X_test)
y_pred[:10]

13/13 [==============================] - 0s 1ms/step


array([[ 8402.916],
       [ 9894.19 ],
       [31858.545],
       [ 3507.734],
       [10104.453],
       [ 4433.332],
       [ 5072.566],
       [ 2969.585],
       [ 7251.29 ],
       [42197.93 ]], dtype=float32)

In [19]:
y_test[:10]

748      8556.90700
745      9910.35985
57      34303.16720
546      3268.84665
279      9855.13140
608      4435.09420
189      4922.91590
507      3077.09550
640      6666.24300
1070    39871.70430
Name: charges, dtype: float64